# n-gram 언어 모델링

In [1]:
import nltk
from nltk.util import ngrams
from collections import Counter

In [4]:
text = "오늘은 날씨가 좋다. 오늘은 기분이 좋다. 오늘은 일이 많다. 오늘은 사람이 많다. 오늘은 날씨가 맑다."

tokens = nltk.word_tokenize(text)

# 1-gram, 2-gram 생성 -> 빈도수 계산
unigrams = tokens
bigrams = list(ngrams(tokens, 2))

unigram_freq = Counter(unigrams)
bigrams_freq = Counter(bigrams)
# print(bigrams_freq)

# 조건부 확률 계산
for (w1, w2), freq in bigrams_freq.items():
    prob = freq / unigram_freq[w1]
    print(f'P({w1}|{w2}) = {prob:.3f}')     # 확률은 w1 뒤에 w2가 올 확률  ('오늘은' 4번 등장, 그 중에 뒤에 '날씨가' 등장한 것은 1번 => 0.25 확률)

P(오늘은|날씨가) = 0.400
P(날씨가|좋다) = 0.500
P(좋다|.) = 1.000
P(.|오늘은) = 0.800
P(오늘은|기분이) = 0.200
P(기분이|좋다) = 1.000
P(오늘은|일이) = 0.200
P(일이|많다) = 1.000
P(많다|.) = 1.000
P(오늘은|사람이) = 0.200
P(사람이|많다) = 1.000
P(날씨가|맑다) = 0.500
P(맑다|.) = 1.000


### Perplexity 계산

In [5]:
import math 

def compute_bigram_perplexity(test_text, unigram_freq, bigrams_freq):
    test_tokens = nltk.word_tokenize(test_text)
    test_bigrams = list(ngrams(test_tokens, 2))

    log_prob_sum = 0
    N = len(test_bigrams)

    for bigram in test_bigrams:
        w1, w2 = bigram
        prob = bigrams_freq.get(bigram, 0) / unigram_freq.get(w1, 1)
        if prob == 0:
            prob = 1e-10
        log_prob_sum += math.log2(prob)

    cross_entropy = -log_prob_sum / N
    perplexity = math.pow(2, cross_entropy)
    return perplexity


In [6]:
train_text = "자연어 처리는 재미있다. 자연어 처리는 어렵지만 도전하고 싶다. 오늘은 날씨가 좋다."

# 단어 토큰화
train_tokens =  nltk.word_tokenize(train_text)

# 유니그램
unigrams = train_tokens

# 바이그램(리스트로 변환)
bigrams = list(ngrams(train_tokens, 2))

# 카운터로 빈도수 카운트(유니그램)
unigram_freq = Counter(unigrams)

# 카운터로 빈도수 카운트(바이그램)
bigram_freq = Counter(bigrams)

In [ ]:
test_sentences = [
    "자연어 처리는 재미있다.",
    "자연어 처리는 어렵지만 도전하고 싶다.",
    "오늘은 날씨가 좋다",
    "기계 번역은 어렵다.",
    "자연어 처리에 도전하고 싶다.",
    "오늘 날씨가 흐리다"
]

for sentence in test_sentences:
    pp = compute_bigram_perplexity(sentence, unigram_freq, bigram_freq)
    print(f'문장: {sentence} | perplexity: {pp:.4f}')

# 앞과 뒤 맥락에 대한 것을 엮어서 이어지는 단어들의 등장확률을 조건부확률로 계산한 것을 기반으로 한다. 

문장: 자연어 처리는 재미있다. | perplexity: 1.2599
문장: 자연어 처리는 어렵지만 도전하고 싶다. | perplexity: 1.1487
문장: 오늘은 날씨가 좋다 | perplexity: 1.0000
문장: 기계 번역은 어렵다. | perplexity: 10000000000.0000
문장: 자연어 처리에 도전하고 싶다. | perplexity: 100000.0000
문장: 오늘 날씨가 흐리다 | perplexity: 10000000000.0000
